In [1]:
!pip install -q langchain transformers tqdm

In [2]:
%reload_ext autoreload
%autoreload 2
%reload_ext dotenv
%dotenv

In [3]:
test_string = "\n\nabc\n"
print(test_string.strip())

abc


In [4]:
import datasets
from tqdm.auto import tqdm
from langchain.docstore.document import Document as LangchainDocument



RAW_KNOWLEDGE_BASE = [
    LangchainDocument(page_content=doc["text"], metadata={"source": doc["source"]}) for doc in tqdm(datasets.load_dataset("m-ric/huggingface_doc", split="train"))
]


  0%|          | 0/2647 [00:00<?, ?it/s]

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
import json
import tiktoken

dataset_collection = []
limit = 10
for chunk_size in [200, 400, 600]:
    for chunk_overlap in [0, int(chunk_size / 2), int(chunk_size / 10)]:
        settings = f"chunk_size:{chunk_size}-chunk_overlap:{chunk_overlap}"
        text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
                model_name="gpt-4",
                chunk_size=chunk_size,
                chunk_overlap=chunk_overlap,
                add_start_index=True,
                strip_whitespace=True,
                separators=["\n\n", "\n", ".", " ", ""],
                disallowed_special=[],
                allowed_special="all"
            )
        print(f"Generating with settings {settings}")
        dataset = {
            "chunk_size": chunk_size, 
            "chunk_overlap": chunk_overlap,
            "limit": limit,
            "texts": [doc.page_content for doc in tqdm(
            text_splitter.split_documents(RAW_KNOWLEDGE_BASE[:limit]))]
        }
        dataset_collection.append(dataset)        
        # tokenizer = tiktoken.encoding_for_model("gpt-4")
        # for text in dataset["texts"]:
        #     print(len(tokenizer.encode(text)))
        
json.dump(dataset_collection, open("./output/huggingface_doc_splits.json", "w"))



In [10]:
content = ""
for doc in RAW_KNOWLEDGE_BASE[:limit]:
    content += doc.page_content
    content += "\n-------------------\n"
    

print(content)
    

 Create an Endpoint

After your first login, you will be directed to the [Endpoint creation page](https://ui.endpoints.huggingface.co/new). As an example, this guide will go through the steps to deploy [distilbert-base-uncased-finetuned-sst-2-english](https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english) for text classification. 

## 1. Enter the Hugging Face Repository ID and your desired endpoint name:

<img src="https://raw.githubusercontent.com/huggingface/hf-endpoints-documentation/main/assets/1_repository.png" alt="select repository" />

## 2. Select your Cloud Provider and region. Initially, only AWS will be available as a Cloud Provider with the `us-east-1` and `eu-west-1` regions. We will add Azure soon, and if you need to test Endpoints with other Cloud Providers or regions, please let us know.

<img src="https://raw.githubusercontent.com/huggingface/hf-endpoints-documentation/main/assets/1_region.png" alt="select region" />

## 3. Define the [Security Level